<h1> NEAT Implementation Practice </h1>
A first-pass attempt at implementing NEAT by translating SethBling's neatevolve.lua script into Python.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math

In [2]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v1' # just need to change this if you wanna try it in environments
env = gym.make(envName)

# print out the observation and action spaces
print(env.action_space.n) # number of valid actions we can take 
print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1



2
4


In [3]:
# hyperparameters, according to section 4.1 Parameter Settings
deltaThreshold = 3.0

c1 = c2 = 1.0
c3 = 0.4
perturbVal = 0.05


numUntilStagnant = 15


genomeMutationChance = 0.8
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.001
smallPopulationNewNodeChance = 0.03
smallPopulationNewConnectionChance = 0.05
largePopulationNewConnectionChance = 0.3

In [4]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [13]:
# genome class 
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    def __init__(self, inputs=1, outputs=1):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False 
        
    def setGenes(self, nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes.copy()
        self.connectGenes = connectGenes.copy()
        
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.setGenes(genome.nodeGenes, genome.connectGenes)
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        # print('Genome global innovation number reset back to: {}'.format(Genome.globalInnovationNumber))
    
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes.clear()
        self.connectGenes.clear()
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node][1] = 0.0
            
    def initEmptyGenome(self):
        self.clearGenome()
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def resetFitness(self):
        self.fitness = 0
        self.sharedFitness = 0
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(outNode=node, inNode=otherNode, weight=random.uniform(-1, 1))
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes and (inNode, outNode) not in self.connectGenes and self.nodeGenes[inNode][0] != 'input' and outNode != inNode): # check that the connection doesnt already exist
                # also dont allow recurrent connections (only feedforward)
                self.connectGenes[(outNode, inNode)] = [Genome.globalInnovationNumber, weight, isExpressed]
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            else: 
                print('Connection already exists. Did not insert')
        else:
            print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
            
    def mutateAddRandomConnection(self):
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    # this is so fucking slow
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output.index(max(output))
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness / speciesCount
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printNodeGeneCount(self):
        print(len(self.nodeGenes))
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

In [ ]:
inputs= env.observation_space.shape[0] + 1
outputs = env.action_space.n
for tries in range(150):
    observation = env.reset()
    genome=Genome(inputs=inputs, outputs=outputs)
    genome.initRandomGenome()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomNode()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    genome.mutateAddRandomConnection()
    t = 0
    while(True):
        env.render()
        action = genome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        genome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            print("Genome fitness: {}".format(genome.fitness))
            break
        t += 1
            
env.close()

In [6]:
# make sure genome 1 is the higher fitness genome 
genome = Genome(inputs, outputs)
Genome.resetGlobalInnovationNumber()
genome.clearGenome()
genome.initRandomGenome()

g1 = Genome.copyGenome(genome)
# g1 = Genome(genome.inputCount, genome.outputCount)
# g1.setGenes(genome.nodeGenes, genome.connectGenes)
g1.fitness = 1.0

g2 = Genome.copyGenome(genome)
# g2 = Genome(genome.inputCount, genome.outputCount)
# g2.setGenes(genome.nodeGenes, genome.connectGenes)
g2.mutateAddRandomNode()
# g2.fitness = 2.0

def crossover(genome1, genome2):
    if (genome2.fitness > genome1.fitness):
        temp = genome1
        genome1 = genome2
        genome2 = temp
    
    g = Genome(genome1.inputCount, genome1.outputCount)
    g1GenomeList = {}
    for connection in genome1.connectGenes:
        #print(genome1.connectGenes[connection])
        g1GenomeList[genome1.connectGenes[connection][0]]=[connection,genome1.connectGenes[connection][1],genome1.connectGenes[connection][2]]
    g2GenomeList = {}
    for connection in genome2.connectGenes:
        g2GenomeList[genome2.connectGenes[connection][0]]=[connection,genome2.connectGenes[connection][1],genome2.connectGenes[connection][2]] 
        
    # when crossing over, the genes in both genomes with the same innovation numbers are lined up
    # in composing the offspring, genes are randomly chosen from either parent at matching genes,
    # whereas all excess or disjoint genes are always included from the more fit parent 
    for i in range(max(len(g1GenomeList), len(g2GenomeList))-1):
        if (i+1 in g1GenomeList and i+1 in g2GenomeList):
            if (random.random() <= 50): # inherit from genome1
                # print('inherited from parent 1')
                g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
            else: # inherit from genome2 
                # print('inherited from parent 2')
                g.connectGenes[g2GenomeList[i+1][0]] = [i+1, g2GenomeList[i+1][1], g2GenomeList[i+1][2]]
            if (g1GenomeList[i+1][2] == False or g1GenomeList[i+1][2] == False):
                if (random.random() <= disableGeneChance):
                    # print('inherited disabled gene')
                    g.connectGenes[g1GenomeList[i+1][0]][2] = False
                else:
                    # print('re-enabled inherited gene')
                    g.connectGenes[g1GenomeList[i+1][0]][2] = True
        if (i+1 in g1GenomeList):
            g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
    
    return g
    
    
g = crossover(g1, g2)
g.printConnectGeneCount()
# def crossover(genome1, genome2):
#     if (genome2.)

added hidden node
10


In [7]:
# TODO: Verify this works properly
def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
    if (not genome1.connectGenes or not genome2.connectGenes):
        return 0
    connectionMismatch = []
    for connection1 in genome1.connectGenes:
        if (connection1 not in genome2.connectGenes):
            connectionMismatch.append(connection1)
    for connection2 in genome2.connectGenes:
        if (connection2 not in genome1.connectGenes):
            connectionMismatch.append(connection2)
    diffsum = 0
    num = 0
    for connection1 in genome1.connectGenes:
        if (connection1 not in connectionMismatch):
            diffsum += abs(genome1.connectGenes[connection1][1] -
                           genome2.connectGenes[connection1][1])
            num += 1
    averageWeightDifference = diffsum / num
    #print(averageWeightDifference)
    N = 0
    if len(genome1.connectGenes) >= len(genome2.connectGenes):
        N = len(genome1.connectGenes)
    else:
        N = len(genome2.connectGenes)
    #print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
    return ((len(connectionMismatch) *
             (c1 + c2)) / N) + (c3 * averageWeightDifference)

# "connection weights mutate as in any NE system, with each connection either perturbed or not at each generation"

In [28]:
max_generations = 1
population_limit = 20

species = {}
pop = {}

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n

for generation in range(max_generations):
    if (generation == 0): # start out with a uniform population of networks with zero hidden networks
        for genome in range(population_limit):
            g = Genome(inputs, outputs)
            Genome.resetGlobalInnovationNumber()
            g.initRandomGenome()
            pop[genome+1] = g
    
    # in each generation, genomes are sequentially placed into species 
    
    fitness = []
    
    for genome in pop:
        if (not species):
            species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter' : 0, 'isStagnant':False,'list':[pop[genome]]}
            pop[genome].species = len(species)
        else:
            for og in species:
                if (delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                    species[og]['list'].append(pop[genome])
                    pop[genome].species = og
                    break
                else:
                    if (og == len(species)-1): # reached the end of the list, and havent matched
                        species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant':False,'list':[pop[genome]]}
        observation = env.reset()
        # evalute the genome's performance 
        t = 0
        while(True):
            env.render()
            action = pop[genome].evaluateAction(observation)
            observation, reward, done, info = env.step(action)
            pop[genome].fitness += reward
            if done:
                # print("Episode finished after {} timesteps".format(t+1))
                fitness.append(pop[genome].fitness)
                if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                    species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                    species[pop[genome].species]['stagnantCounter'] = 0
                    species[pop[genome].species]['isStagnant'] = False
                # print("Genome fitness: {}".format(pop[genome].fitness))
                break
                
    
    
    
    print('Number of species at generation {}: {}'.format(generation+1, len(species)))
    print('highest achieved score in this generation {}'.format(max(fitness)))
     
    # print(species[1])
    adjustedFitness = []
    for genome in pop:
        pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
        adjustedFitness.append(pop[genome].sharedFitness)
        species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
        if pop[genome].sharedFitness > species[pop[genome].species]['maxAdjustedFitness']:
            species[pop[genome].species]['maxAdjustedFitness'] = pop[genome].sharedFitness
            
    for s in species:
        print('Maximum adjusted fitness in species {}: {}'.format(s, species[s]['maxAdjustedFitness']))
        print('Summed adjusted fitness in species {}: {}'.format(s, species[s]['summedSharedFitness']))
        
        
        
    # each existing species is represented by a random genome inside the species from the previous generation 
    if (generation > 0):
        for s in species:
            species[s] = {'summedSharedFitness': 0.0, 'maxFitness': species[s]['maxFitness'], 'maxAdjustedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s])]}
            if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                species[s]['isStagnant'] = True
    
    
    pop.clear()
    # the champion of each species with more than five networks was copied into the next generation unchanged 
    for s in species:
        if 
    
    
env.close()

Number of species at generation 1: 1
highest achieved score in this generation 87.0
Maximum adjusted fitness in species 1: 14.5
Summed adjusted fitness in species 1: 65.33333333333331


In [ ]:
maxNodes = 100000
max_generations = 150
population = 150

globalInnovation = 1

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n
# TODO: consider making these into dictionaries ?
for generation in range(max_generations):
    print('Starting new generation: {}'.format(generation))
    species = {}
    speciesCount = 0
    pop = {}
    for genome in range(population):
        # i think what I'll attempt first is 
        # create the genome, then subject it to all of the random chances 
        g = Genome()
        # if this is the first generation, then just initRandomGenome
        if (generation == 1):
            g.initRandomGenome(inputs, outputs)
        else: # else, initialize the genome from something else (?) have trouble with this in terms of cross over and stuff
            # TODO: replace this 
            g.initRandomGenome(inputs, outputs)
        # insert all of the random chances here 
        if (g.connectGenes): # if there are any connections
            if (random.random() <= genomeMutationChance):
                for connection in g.connectGenes:
                    if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                        g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                    else:
                        g.connectGenes[connection][1] = random.random() # set a new value for 
            
                
            
        pop[genome] = g # add the genome into the current population 
        
        if (not species): # if there are no species in the species list, initialize it with the first genome created
            print('Species list empty. Creating new species')
            species[speciesCount] = [g]
            speciesCount += 1
        else:
            for og in species: # for all of the species in the species list 
                if (delta(g, species[og][0], c1, c2, c3) > deltaThreshold): # species[og][0] gets the first genome of a species
                    print('DeltaThreshold surpassed. Creating new species')
                    species[speciesCount] = g 
                    speciesCount += 1 
                else:
                    print('Adding genome into species#{}'.format(og))
                    species[og].append(g)
                    g.species = og
                    break
                    
    print("Number of different species in generation {}: {}".format(generation, len(species)))

# for generation in range(max_generations):
#     species = []
#     pop= []
#     for genome in range(population):
#         g = Genome()
#         g.initRandomGenome()
#         pop.append(g)
        
#         if (not species):
#             species.append(g)
#         else:
#             for og in species:
#                 if (delta(g, og, c1, c2, c3) > deltaThreshold):
#                     species.append(g)
#                 else:
#                     g.species = species.index(og)

                    
                    
#     print("Number of different species in generation {}: {}".format(generation, len(species)))
